In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
## bUILT-iN pACKAGES
import sys, os, time, bz2, zlib, pickle, math, json, csv
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
np.set_printoptions(suppress=True)
%matplotlib inline
from IPython.display import display

## lAST sUMMER
from romtoslp import *
from sentences import *
from DCS import *
import MatDB
from heap_n_PrimMST import *
from ECL_MST import *

## lAST yEAR
# from word_definite import *
from nnet import *
# from heap_n_PrimMST import *
# from word_definite import *

In [ ]:
import winsound
def playBeep():
    for i in range(3):
        winsound.Beep(2200, 300)
        winsound.Beep(2600, 300)

In [3]:
loaded_SKT = pickle.load(open('../Simultaneous_CompatSKT_10K.p', 'rb'), encoding=u'utf-8')
loaded_DCS = pickle.load(open('../Simultaneous_DCS_10K.p', 'rb'), encoding=u'utf-8')

In [4]:
# dataset_4k_1k = pickle.load(open('../SmallDataset_4K_1K.p', 'rb'))
# TrainFiles = dataset_4k_1k['TrainFiles']
# TestFiles = dataset_4k_1k['TestFiles']

# dataset_6k_3k = pickle.load(open('../SmallDataset_6K_3K.p', 'rb'))
# TrainFiles_2 = dataset_6k_3k['TrainFiles']
# TestFiles_2 = dataset_6k_3k['TestFiles']

In [5]:
# matDB = MatDB.MatDB()

In [4]:
# Create Training File List
excluded_files = []
with open('inputs/Baseline4_advSample.csv', 'r') as f_handle:
    opener = csv.reader(f_handle)
    for line in opener:
        excluded_files.append(line[1].replace('.p', '.ds.bz2'))

bz2_input_folder = '../NewData/skt_dcs_DS.bz2_1L_bigram_10K_d2K/'
# bz2_input_folder = '/home/rs/15CS91R05/vishnu/Data/skt_dcs_DS.bz2_compat_10k_check//'
all_files = []
skipped = 0
for f in os.listdir(bz2_input_folder):
    if '.ds.bz2' in f:
        if f in excluded_files:
            skipped += 1
            continue
        all_files.append(f)

print(skipped, 'files will not be used for training')
print('Size of training set:', len(all_files))

TrainFiles = all_files        

957 files will not be used for training
Size of training set: 8219


In [5]:
def open_dsbz2(filename):
    with bz2.BZ2File(filename, 'r') as f:
        loader = pickle.load(f)
    
    conflicts_Dict_correct = loader['conflicts_Dict_correct']
    nodelist_to_correct_mapping = loader['nodelist_to_correct_mapping']
    nodelist_correct = loader['nodelist_correct']
    featVMat_correct = loader['featVMat_correct']
    featVMat = loader['featVMat']
    conflicts_Dict = loader['conflicts_Dict']
    nodelist = loader['nodelist']
    
    return (nodelist_correct, conflicts_Dict_correct, featVMat_correct, nodelist_to_correct_mapping,\
            nodelist, conflicts_Dict, featVMat)


In [6]:
# from MatDB import *
import word_definite as WD
from heap_n_PrimMST import *
from nnet import *

In [7]:
"""
################################################################################################
######################  CREATE SEVERAL DATA STRUCTURES FROM SENTENCE/DCS  ######################
###########################  NODELIST, ADJACENCY LIST, GRAPH, HEAP #############################
################################################################################################
"""
def GetTrainingKit(sentenceObj, dcsObj):
    nodelist = GetNodes(sentenceObj)
    
    # Nodelist with only the correct_nodes
    nodelist2 = GetNodes(sentenceObj)
    nodelist2_to_correct_mapping = {}
    nodelist_correct = []
    search_key = 0
    first_key = 0
    for chunk_id in range(len(dcsObj.lemmas)):
        while nodelist2[first_key].chunk_id != chunk_id:
            first_key += 1
        for j in range(len(dcsObj.lemmas[chunk_id])):
            search_key = first_key
            while (nodelist2[search_key].lemma != rom_slp(dcsObj.lemmas[chunk_id][j])) or (nodelist2[search_key].cng != dcsObj.cng[chunk_id][j]):
                search_key += 1
                if search_key >= len(nodelist2) or nodelist2[search_key].chunk_id > chunk_id:
                    break
    #         print((rom_slp(dcsObj.lemmas[chunk_id][j]), dcsObj.cng[chunk_id][j]))
    #         print(nodelist[search_key])
            nodelist2_to_correct_mapping[len(nodelist_correct)] = search_key
            nodelist_correct.append(nodelist2[search_key])
    return (nodelist, nodelist_correct, nodelist2_to_correct_mapping)
    

def GetGraph(nodelist, neuralnet):
    if not neuralnet.outer_relu:
        conflicts_Dict = Get_Conflicts(nodelist)

        featVMat = Get_Feat_Vec_Matrix(nodelist, conflicts_Dict)

        (WScalarMat, SigmoidGateOutput) = Get_W_Scalar_Matrix_from_FeatVect_Matrix(featVMat, nodelist, conflicts_Dict, neuralnet)
        return (conflicts_Dict, featVMat, WScalarMat, SigmoidGateOutput)
    else:
        conflicts_Dict = Get_Conflicts(nodelist)

        featVMat = Get_Feat_Vec_Matrix(nodelist, conflicts_Dict)

        WScalarMat = Get_W_Scalar_Matrix_from_FeatVect_Matrix(featVMat, nodelist, conflicts_Dict, neuralnet)
        return (conflicts_Dict, featVMat, WScalarMat)

In [8]:
trainingStatus = defaultdict(lambda: bool(False))

In [9]:
"""
################################################################################################
##############################  TRAIN FUNCTION  ################################################
################################################################################################
"""

def train_generator(loaded_SKT, loaded_DCS, bz2_input_folder, n_trainset = -1, iterationPerBatch = 10, filePerBatch = 20, _debug = True):
    # Train
    if n_trainset == -1:
        n_trainset = len(TrainFiles)
        totalBatchToTrain = math.ceil(n_trainset/filePerBatch)
    else:
        totalBatchToTrain = math.ceil(n_trainset/filePerBatch)
    

    for iterout in range(totalBatchToTrain):
        # Add timer
        startT = time.time()

        # Change current batch
        trainer.Save(p_name)
        print('Batch: ', iterout)
        files_for_batch = TrainFiles[iterout*filePerBatch:(iterout + 1)*filePerBatch]
        print(files_for_batch)
        # trainer.Load('outputs/neuralnet_trained.p')
        try:
            # Run few times on same set of files
            for iterin in range(iterationPerBatch):
                print('ITERATION IN', iterin)        
                for fn in files_for_batch:
                    trainFileName = fn.replace('.ds.bz2', '.p2')
                    sentenceObj = loaded_SKT[trainFileName]
                    dcsObj = loaded_DCS[trainFileName]
                    if trainingStatus[sentenceObj.sent_id]:
                        continue
                    # trainer.Save('outputs/saved_trainer.p')
                    try:
                        trainer.Train(sentenceObj, dcsObj, bz2_input_folder, _debug)
                    except (IndexError, KeyError) as e:
                        print('\x1b[31mFailed: {} \x1b[0m'.format(sentenceObj.sent_id))
                sys.stdout.flush() # Flush IO buffer 
            finishT = time.time()
            print('Avg. time taken by 1 file(1 iteration): {:.3f}'.format((finishT - startT)/(iterationPerBatch*filePerBatch)))
        except KeyboardInterrupt:
            print('Training paused')
            trainer.Save(p_name)
            yield 'Hi'
    trainer.Save(p_name)
    
    sys.stdout.flush() # Flush IO buffer 
                
def test(loaded_SKT, loaded_DCS, n_testSet = -1, _testFiles = None, n_checkpt = 100):
    total_lemma = 0;
    correct_lemma = 0;

    total_word = 0;
    total_output_nodes = 0
    correct_word = 0;
    file_counter = 0
    if _testFiles is None:
        if n_testSet == -1:
            _testFiles = TestFiles
        else:
            _testFiles = TestFiles[0:n_testSet]
    else:
        if n_testSet == -1:
            _testFiles = _testFiles
        else:
            _testFiles = _testFiles[0:n_testSet]
            
    recalls = []
    recalls_of_word = []
    precisions = []
    precisions_of_words = []
    for fn in _testFiles:
        if file_counter % n_checkpt == 0:
            print(file_counter,' Checkpoint... ')
            if file_counter > 0:
                print('Avg. Micro Recall of Lemmas: {}'.format(np.mean(np.array(recalls))))
                print('Avg. Micro Recall of Words: {}'.format(np.mean(np.array(recalls_of_word))))
                print('Avg. Micro Precision of Lemmas: {}'.format(np.mean(np.array(precisions))))
                print('Avg. Micro Precision of Words: {}'.format(np.mean(np.array(precisions_of_words))))
            sys.stdout.flush() # Flush IO buffer 
        
        file_counter += 1
        
        testFileName = fn.replace('.ds.bz2', '.p2')
        sentenceObj = loaded_SKT[testFileName]
        dcsObj = loaded_DCS[testFileName]    
        
        try:
            (word_match, lemma_match, n_dcsWords, n_output_nodes) = trainer.Test(sentenceObj, dcsObj)
            
            recalls.append(lemma_match/n_dcsWords)
            recalls_of_word.append(word_match/n_dcsWords)
            
            precisions.append(lemma_match/n_output_nodes)
            precisions_of_words.append(word_match/n_output_nodes)
            
            total_lemma += n_dcsWords
            total_word += n_dcsWords
            
            total_output_nodes += n_output_nodes            
            
            correct_lemma += lemma_match
            correct_word += word_match
        except (IndexError, KeyError) as e:
            print('Failed!')        

    print('Avg. Micro Recall of Lemmas: {}'.format(np.mean(np.array(recalls))))
    print('Avg. Micro Recall of Words: {}'.format(np.mean(np.array(recalls_of_word))))
    print('Avg. Micro Precision of Lemmas: {}'.format(np.mean(np.array(precisions))))
    print('Avg. Micro Precision of Words: {}'.format(np.mean(np.array(precisions_of_words))))
    
    return (recalls, recalls_of_word, precisions, precisions_of_words)
    

In [10]:
# NOW-OLD FUNCTION
# def GetLoss(_mst_adj_graph, _mask_de_correct_edges, _negLogLikelies):
#     _negLogLikelies = _negLogLikelies.copy()
#     _negLogLikelies[~_mst_adj_graph] = 0
#     _negLogLikelies[~_mask_de_correct_edges] *= -1 # BAKA!!! Check before you try to fix this again
#     return np.sum(_negLogLikelies)    

# NEW FUNCTION
def GetLoss(_mst_adj_graph, _mask_de_correct_edges, _WScalarMat):
    _WScalarMat = _WScalarMat.copy()
    _WScalarMat[_mst_adj_graph&(~_mask_de_correct_edges)] *= -1 # BAKA!!! Check before you try to fix this again
    _WScalarMat[~_mst_adj_graph] = 0
    return np.sum(_WScalarMat)

In [11]:
"""
################################################################################################
##############################  GET A FILENAME TO SAVE WEIGHTS  ################################
################################################################################################
"""
import time
st = str(int((time.time() * 1e6) % 1e13))
log_name = 'logs/train_nnet_t{}.out'.format(st)
p_name = 'outputs/train_nnet_t{}.p'.format(st)
print('nEURAL nET wILL bE sAVED hERE: ', p_name)

nEURAL nET wILL bE sAVED hERE:  outputs/train_nnet_t1568073772239.p


In [12]:
class Trainer:
    def __init__(self, modelFile = None):
        if modelFile is None:
            self.hidden_layer_size = 1800
            self._edge_vector_dim = 2000
            # self._edge_vector_dim = WD._edge_vector_dim
            # self._full_cnglist = list(WD.mat_cngCount_1D)
            
            self.neuralnet = NN(self._edge_vector_dim, self.hidden_layer_size, outer_relu=True)
            self.history = defaultdict(lambda: list())
        else:
            loader = pickle.load(open(filename, 'rb'))
            
            self.neuralnet.hidden_layer_size = loader['n']
            self.neuralnet._edge_vector_dim = loader['d']

            self.neuralnet = NN(self._edge_vector_dim, self.hidden_layer_size, outer_relu=True)

            self.neuralnet.U = loader['U']
            self.neuralnet.W = loader['W']
            self.neuralnet.B1 = loader['B1']
            self.neuralnet.B2 = loader['B2']
            
            self.history = defaultdict(lambda: list())
            
        # SET LEARNING RATES
        self.neuralnet.etaW = 3e-4
        self.neuralnet.etaB1 = 1e-4
        
        self.neuralnet.etaU = 1e-4
        self.neuralnet.etaB2 = 1e-4
            
    def Reset(self):
        self.neuralnet = NN(self._edge_vector_dim, self.hidden_layer_size)
        self.history = defaultdict(lambda: list())
        
    def Save(self, filename):
        print('Weights Saved: ', p_name)
        pickle.dump({
                'U': self.neuralnet.U,
                'W': self.neuralnet.W,
                'n': self.neuralnet.n,
                'd': self.neuralnet.d,
                'B1': self.neuralnet.B1,
                'B2': self.neuralnet.B2
            }, open(p_name, 'wb'))
        return
        
    
    def Load(self, filename):
        loader = pickle.load(open(filename, 'rb'))
        self.neuralnet.U = loader['U']
        self.neuralnet.W = loader['W']
        self.neuralnet.B1 = loader['B1']
        self.neuralnet.B2 = loader['B2']
        self.neuralnet.hidden_layer_size = loader['n']
        self.neuralnet._edge_vector_dim = loader['d']
        
    def Test(self, sentenceObj, dcsObj, dsbz2_name):
        neuralnet = self.neuralnet
        minScore = np.inf
        minMst = None
        
        # dsbz2_name = sentenceObj.sent_id + '.ds.bz2'
        (nodelist_correct, conflicts_Dict_correct, featVMat_correct, nodelist_to_correct_mapping,\
            nodelist, conflicts_Dict, featVMat) = open_dsbz2(dsbz2_name)
        
        # if len(nodelist) > 50:
        #     return None

        if not self.neuralnet.outer_relu:
            (WScalarMat, SigmoidGateOutput) = Get_W_Scalar_Matrix_from_FeatVect_Matrix(featVMat, nodelist, conflicts_Dict, neuralnet)
        else:
            WScalarMat = Get_W_Scalar_Matrix_from_FeatVect_Matrix(featVMat, nodelist, conflicts_Dict, neuralnet)
        
        # print('NeuralNet Time: ', time.time() - startT)
        # startT = time.time()
        
        # Get all MST
        for source in range(len(nodelist)):
            #(mst_nodes, mst_adj_graph, _) = MST(nodelist, WScalarMat, conflicts_Dict, source)
            (mst_nodes_bool,mst_nodes,mst_adj_graph)=MST_ECL(nodelist,WScalarMat,conflicts_Dict,source)
            # print('.', end = '')
            score = GetMSTWeight(mst_adj_graph, WScalarMat)
            if(score < minScore):
                minScore = score
                minMst = mst_nodes
        dcsLemmas = [[rom_slp(l) for l in arr]for arr in dcsObj.lemmas]
        word_match = 0
        lemma_match = 0
        n_output_nodes = 0
        for chunk_id, wdSplit in minMst.items():
            for wd in wdSplit:
                n_output_nodes += 1
                # Match lemma
                search_result = [i for i, j in enumerate(dcsLemmas[chunk_id]) if j == wd.lemma]
                if len(search_result) > 0:
                    lemma_match += 1
                # Match CNG
                for i in search_result:
                    if(dcsObj.cng[chunk_id][i] == str(wd.cng)):
                        word_match += 1
                        # print(wd.lemma, wd.cng)
                        break
        dcsLemmas = [l for arr in dcsObj.lemmas for l in arr]
        
        # print('All MST Time: ', time.time() - startT)
        # print('Node Count: ', len(nodelist))
#         print('\nFull Match: {}, Partial Match: {}, OutOf {}, NodeCount: {}, '.\
#               format(word_match, lemma_match, len(dcsLemmas), len(nodelist)))
        return (word_match, lemma_match, len(dcsLemmas), n_output_nodes)
    
    def Train(self, sentenceObj, dcsObj, bz2_input_folder, _debug = True):
        # Hyperparameter for hinge loss: m
        m_hinge_param = 14
        
        dsbz2_name = sentenceObj.sent_id + '.ds.bz2'
        (nodelist_correct, conflicts_Dict_correct, featVMat_correct, nodelist_to_correct_mapping,\
            nodelist, conflicts_Dict, featVMat) = open_dsbz2(bz2_input_folder + dsbz2_name)
        # Train for large graphs separately
#         if len(nodelist) < 40:
#             return
        
        """ FORM MAXIMUM(ENERGY) SPANNING TREE OF THE GOLDEN GRAPH : WORST GOLD STRUCTURE """
        WScalarMat_correct = Get_W_Scalar_Matrix_from_FeatVect_Matrix(featVMat_correct, nodelist_correct,\
                                                                      conflicts_Dict_correct, self.neuralnet)
        source = 0
        """ Find the max spanning tree : negative Weight matrix passed """
#         (max_st_gold_ndict, max_st_adj_gold_small, _) =\
#             MST(nodelist_correct, -WScalarMat_correct, conflicts_Dict_correct, source)
        (max_st_gold_ndict, max_st_adj_gold_small, _) =\
            MST(nodelist_correct, -WScalarMat_correct, conflicts_Dict_correct, source)
        energy_gold_max_ST = np.sum(WScalarMat_correct[max_st_adj_gold_small])
        
        """ Convert correct spanning tree graph adj matrix to full marix dimensions """
        """ Create full-size adjacency matrix for correct_mst_small """
        nodelen = len(nodelist)
        max_st_adj_gold = np.ndarray((nodelen, nodelen), np.bool)*False # T_STAR
        for i in range(max_st_adj_gold_small.shape[0]):
            for j in range(max_st_adj_gold_small.shape[1]):
                max_st_adj_gold[nodelist_to_correct_mapping[i], nodelist_to_correct_mapping[j]] =\
                    max_st_adj_gold_small[i, j]
        
        """ Delta(Margin) Function : MASK FOR WHICH NODES IN NODELIST BELONG TO DCS """
        gold_nodes_mask = np.array([False]*len(nodelist))
        gold_nodes_mask[list(nodelist_to_correct_mapping.values())] = True
        margin_f = lambda nodes_mask: np.sum(nodes_mask&(~gold_nodes_mask))**2
        
        """ FOR ALL POSSIBLE MST FROM THE COMPLETE GRAPH """
        WScalarMat = Get_W_Scalar_Matrix_from_FeatVect_Matrix(featVMat, nodelist, conflicts_Dict, self.neuralnet)

        """ For each node - Find MST with that source"""
        min_STx = None # Min Energy spanning tree with worst margin with gold_STx
        min_marginalized_energy = np.inf
        
        # Generate random set of nodes from which mSTs are to be considered
        n_nodes = len(nodelist)
        selection_prob = 0.4
        select_flag = np.random.rand(n_nodes) < selection_prob
        # Fix if all zeros
        if np.sum(select_flag) == 0:
            select_flag[np.random.randint(n_nodes)] = 1
        
        for source in range(len(nodelist)):
            (mst_nodes, mst_adj_graph, mst_nodes_bool) = MST(nodelist, WScalarMat, conflicts_Dict, source) # T_X
            # print('.', end = '')
           
            marginalized_en = np.sum(WScalarMat[mst_adj_graph]) - margin_f(mst_nodes_bool)
            # Minimum marginalized spanning tree : Randomization applied
            # if marginalized_en < min_marginalized_energy and select_flag[source]:
            if marginalized_en < min_marginalized_energy:
                min_marginalized_energy = marginalized_en
                min_STx = mst_adj_graph
            # Energy diff should all be negative
            if _debug:
                print('Source: [{}], Node_Diff:{}, Max_Gold_En: {:.3f}, Energy: {:.3f}'.\
                      format(source, np.sum((~gold_nodes_mask)&mst_nodes_bool), energy_gold_max_ST,  np.sum(WScalarMat[mst_adj_graph])))

        """ Gradient Descent """
        # FOR MOST OFFENdING Y
        doBpp = False
        
        Total_Loss = energy_gold_max_ST - min_marginalized_energy
        if Total_Loss > 0:
            dLdOut = np.zeros_like(WScalarMat)
            dLdOut[max_st_adj_gold&(~min_STx)] = 1
            dLdOut[(~max_st_adj_gold)&min_STx] = -1
            if _debug:
                print('{}. '.format(sentenceObj.sent_id), end = '')
            self.neuralnet.Back_Prop(dLdOut, len(nodelist), featVMat, _debug)
        else:
            trainingStatus[sentenceObj.sent_id] = True
        if _debug:
            print("\nFileKey: %s, Loss: %6.3f" % (sentenceObj.sent_id, Total_Loss))

In [13]:
# trainer.neuralnet.U

In [14]:
trainer = None
def InitModule():
    global trainer
    trainer = Trainer()
InitModule()
trainingStatus = defaultdict(lambda: bool(False))
# trainer.Load('outputs/train_nnet_t526539685574.p')

In [ ]:
"""
################################################################################################
################################################################################################
################################################################################################
"""
# if __name__ == '__main__':
# loaded_SKT = pickle.load(open('../Simultaneous_CompatSKT_10K.p', 'rb'))
# loaded_DCS = pickle.load(open('../Simultaneous_DCS_10K.p', 'rb'))
# main(loaded_SKT, loaded_DCS)
np.set_printoptions(suppress=False)

fn = TrainFiles[5].replace('.ds.bz2', '.p2')
print(fn)
for _ in range(1):
    trainer.Train(loaded_SKT[fn], loaded_DCS[fn], bz2_input_folder, _debug = True)

# # fn = TestFiles[1].replace('.ds.bz2', '.p2')
# trainer.Test(loaded_SKT[fn], loaded_DCS[fn])
# print ("Not Implemented")

In [15]:
train = train_generator(loaded_SKT, loaded_DCS, bz2_input_folder, n_trainset = -1, filePerBatch = 10, iterationPerBatch = 5, _debug=False)

In [ ]:
# Complete Training
# tips: try increasing iterations per batch
trainingStatus = defaultdict(lambda: bool(False)) # Reset it after 3 epochs of full-training set
train.__next__()

Weights Saved:  outputs/train_nnet_t1543721276199.p
Batch:  0
['10250.ds.bz2', '428839.ds.bz2', '20035.ds.bz2', '32130.ds.bz2', '418916.ds.bz2', '154658.ds.bz2', '65093.ds.bz2', '266079.ds.bz2', '429657.ds.bz2', '389979.ds.bz2']
ITERATION IN 0
ITERATION IN 1
ITERATION IN 2
ITERATION IN 3
ITERATION IN 4
Avg. time taken by 1 file(1 iteration): 0.966
Weights Saved:  outputs/train_nnet_t1543721276199.p
Batch:  1
['365964.ds.bz2', '29623.ds.bz2', '162846.ds.bz2', '296029.ds.bz2', '338298.ds.bz2', '187738.ds.bz2', '171533.ds.bz2', '310155.ds.bz2', '22207.ds.bz2', '12964.ds.bz2']
ITERATION IN 0
ITERATION IN 1
ITERATION IN 2
ITERATION IN 3
ITERATION IN 4
Avg. time taken by 1 file(1 iteration): 2.539
Weights Saved:  outputs/train_nnet_t1543721276199.p
Batch:  2
['231593.ds.bz2', '399906.ds.bz2', '194861.ds.bz2', '130103.ds.bz2', '433682.ds.bz2', '164353.ds.bz2', '35530.ds.bz2', '240430.ds.bz2', '37993.ds.bz2', '57802.ds.bz2']
ITERATION IN 0
ITERATION IN 1
ITERATION IN 2
ITERATION IN 3
ITERATION

ITERATION IN 1
ITERATION IN 2
ITERATION IN 3
ITERATION IN 4
Avg. time taken by 1 file(1 iteration): 1.626
Weights Saved:  outputs/train_nnet_t1543721276199.p
Batch:  24
['407205.ds.bz2', '331734.ds.bz2', '7608.ds.bz2', '397770.ds.bz2', '9174.ds.bz2', '16297.ds.bz2', '281361.ds.bz2', '358204.ds.bz2', '179227.ds.bz2', '205930.ds.bz2']
ITERATION IN 0
ITERATION IN 1
ITERATION IN 2
ITERATION IN 3
ITERATION IN 4
Avg. time taken by 1 file(1 iteration): 3.363
Weights Saved:  outputs/train_nnet_t1543721276199.p
Batch:  25
['24816.ds.bz2', '32806.ds.bz2', '396217.ds.bz2', '390969.ds.bz2', '28467.ds.bz2', '2777.ds.bz2', '25953.ds.bz2', '45790.ds.bz2', '30615.ds.bz2', '159020.ds.bz2']
ITERATION IN 0
ITERATION IN 1
ITERATION IN 2
ITERATION IN 3
ITERATION IN 4
Avg. time taken by 1 file(1 iteration): 3.929
Weights Saved:  outputs/train_nnet_t1543721276199.p
Batch:  26
['273342.ds.bz2', '366430.ds.bz2', '383693.ds.bz2', '337041.ds.bz2', '74230.ds.bz2', '25598.ds.bz2', '378810.ds.bz2', '272439.ds.bz2',

Batch:  47
['144531.ds.bz2', '126863.ds.bz2', '433738.ds.bz2', '441056.ds.bz2', '102862.ds.bz2', '157559.ds.bz2', '108861.ds.bz2', '322071.ds.bz2', '44304.ds.bz2', '37291.ds.bz2']
ITERATION IN 0
ITERATION IN 1
ITERATION IN 2
ITERATION IN 3
ITERATION IN 4
Avg. time taken by 1 file(1 iteration): 2.866
Weights Saved:  outputs/train_nnet_t1543721276199.p
Batch:  48
['21410.ds.bz2', '421530.ds.bz2', '212722.ds.bz2', '28812.ds.bz2', '328144.ds.bz2', '46158.ds.bz2', '330967.ds.bz2', '352631.ds.bz2', '332137.ds.bz2', '256951.ds.bz2']
ITERATION IN 0
ITERATION IN 1
ITERATION IN 2
ITERATION IN 3
ITERATION IN 4
Avg. time taken by 1 file(1 iteration): 2.454
Weights Saved:  outputs/train_nnet_t1543721276199.p
Batch:  49
['19533.ds.bz2', '25499.ds.bz2', '2669.ds.bz2', '22782.ds.bz2', '166813.ds.bz2', '320985.ds.bz2', '84313.ds.bz2', '333611.ds.bz2', '282848.ds.bz2', '376933.ds.bz2']
ITERATION IN 0
ITERATION IN 1
ITERATION IN 2
ITERATION IN 3
ITERATION IN 4
Avg. time taken by 1 file(1 iteration): 1.94

In [ ]:
# Test on training set
test(loaded_SKT, loaded_DCS, n_testSet=100, _testFiles=TrainFiles, n_checkpt = 10)
playBeep()

In [56]:
# POST TEST 1K SET
_ = test(loaded_SKT, loaded_DCS, n_testSet = -1, _testFiles = TestFiles, n_checkpt = 50)
playBeep()

0  Checkpoint... 
50  Checkpoint... 
100  Checkpoint... 
150  Checkpoint... 
200  Checkpoint... 
250  Checkpoint... 
300  Checkpoint... 
350  Checkpoint... 
400  Checkpoint... 
450  Checkpoint... 
500  Checkpoint... 
550  Checkpoint... 
600  Checkpoint... 
650  Checkpoint... 
Avg. Micro Recall of Lemmas: 0.8369512331454708
Avg. Micro Recall of Words: 0.6885798895409492
Avg. Micro Precision of Lemmas: 0.78979303122867
Avg. Micro Precision of Words: 0.6511872539596757


In [ ]:
# Test on Larger Set
_ = test(loaded_SKT, loaded_DCS, n_testSet = 3000, _testFiles = TestFiles_2, n_checkpt = 100)
playBeep()

Weights Saved:  outputs/train_nnet_t526539685574.p


In [ ]:
trainer.neuralnet.W

In [ ]:
trainer.neuralnet.U

In [15]:
for fn in TrainFiles[:10]:
    dsbz2_name = bz2_input_folder + fn
    fn = fn.replace('.ds.bz2', '.p2')
    print(fn)
    ret = trainer.Test(loaded_SKT[fn], loaded_DCS[fn], dsbz2_name)
    break

10250.p2


In [15]:
WD.word_definite_extInit(matDB)
# node2 = WD.word_definite('tIkzRam', 'tIkzRa', 31, 0, 0)
node1 = WD.word_definite('Adi', 'Adi', 31, 0, 0)
node2 = WD.word_definite('ca', 'ca', 2, 0, 1)
# node1 = WD.word_definite('koRam', 'koRa', 31, 0, 1)
print(node1)
print(node2)

feats = WD.Get_Features(node1, node2)
print(feats.shape)

WD_Node[C: 0, P: 0, Adi @(31) => Adi]
WD_Node[C: 1, P: 0, ca @(2) => ca]
(1000, 1)


In [25]:
Wmat 

array([[ 0.        ,  0.44093286,  0.54536647,  0.26590788,  0.89111612,
         0.33948032, -0.03414859],
       [ 0.84916065,  0.        ,  1.1908067 ,  1.00455444,  1.54897616,
         0.94725297,  1.38592746],
       [-0.40401637, -0.45824241,  0.        , -0.66569737,  0.02653288,
        -0.6492698 , -0.28316029],
       [-0.28461703, -0.35128149, -0.57814776,  0.        ,  0.53596003,
        -0.61933034, -0.76127807],
       [ 0.50401231,  0.48448779,  1.19736903,  0.747994  ,  0.        ,
         0.75652086,  1.44398605],
       [ 0.22824487,  0.24474065,  0.41312113,  0.37012238,  0.5754286 ,
         0.        ,  0.5124145 ],
       [-0.39726662, -0.46351106, -0.49387717, -0.53360749, -0.06269749,
        -0.61960008,  0.        ]])

In [16]:
ret

(2, 2, 4, 8)

######   THE END
-----------------------------------------------------------------------